In [ ]:
from google.colab import drive; drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/amex_data_set/

/content/drive/MyDrive/amex_data_set


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import gc
import os
import joblib
import random
import warnings
import itertools
import scipy as sp
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
from itertools import combinations
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import LabelEncoder
import warnings; warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold, train_test_split

import sklearn
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer  
from sklearn.preprocessing import PowerTransformer

import tensorflow as tf
device_name = tf.test.gpu_device_name()

In [ ]:
def elbo(df, start, stop, seed):
  x_cols = df.values
  ss = []
  k_vals = range(start, stop)#(2, 12)

  for k in k_vals:
    seed = k
    km = KMeans(n_clusters=k, random_state=seed)
    km.fit(x_cols)
    ss.append(km.inertia_)

  plt.plot(k_vals, ss, '-o')
  plt.xlabel('k clusters')
  plt.ylabel('sum-of-squared errors')
  plt.title('elbow method')
  plt.show()

def fillnans(df, cols, filler):#np.mean
  for c in df[cols].columns:
    thresh = (len(df[c]) - df[c].isna().sum())/len(df[c])
    if thresh < 0.7:
      df.drop(c, axis=1, inplace=True)
    elif filler == 'zero':
      df[c].fillna(0, inplace=True)
    elif filler == 'mean':
      df[c].fillna(np.mean(df[c]), inplace=True)
    elif filler == 'min':
      df[c].fillna(df[c].min()*2, inplace=True)

def minLst(dfList):
  min = 0
  minList = []
  for df in dfList:
    for c in df.columns:
      if np.min(df[c]) <= 0:
        tmp = df[c].min()
        if tmp < min: 
          min = tmp
    minList.append(min)
    print(c, ' : ', min)
    min = 0
  return minList

def floats(df):#AND converts float cols to str
  floats = []
  count = 0
  for c in df.columns:
    if 'float' in str(df[c].dtypes):
      floats.append(c)
  return floats

def get_difference(data, num_features):
  df1 = []
  customer_ids = []
  
  for customer_id, df in tqdm(data.groupby(['customer_ID'])):
      diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
      df1.append(diff_df1)
      customer_ids.append(customer_id)
  
  df1 = np.concatenate(df1, axis = 0)
  df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
  df1['customer_ID'] = customer_ids
  return df1

def preprocessor(df, km, cluster_specifier):
  scaler = StandardScaler()
  df_scaled = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
  pca = PCA(n_components=round((df_scaled.shape[1])*0.8))
  df_scaled_pca = pd.DataFrame(pca.fit_transform(df_scaled), index=df_scaled.index)
  km.fit(df_scaled_pca.values)
  predict = km.predict(df_scaled_pca)
  df_scaled_pca[cluster_specifier + '_clus'] = pd.Series(predict, index=df_scaled_pca.index)
  return df_scaled_pca

def listOfLists(df):
  bList = []
  dList = []
  pList = []
  rList = []
  sList = []
  for c in df.columns:
    if 'B_' in c:
      bList.append(c)
    if 'D_' in c:
      dList.append(c)
    if 'P_' in c:
      pList.append(c)
    if 'R_' in c:
      rList.append(c)
    if 'S_' in c and c is not 'S_2':
      sList.append(c)
  return bList, dList, pList, rList, sList

In [ ]:
class CFG:
    seed = 42
    n_folds = 5
    target = 'target'
    input_dir = '/content/drive/MyDrive/amex_data_set'

    def seed_everything(seed):
        random.seed(seed)
        np.random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)

    def read_data():
        train = pd.read_csv(CFG.input_dir + 'train_fe_pp.csv')
        test = pd.read_csv(CFG.input_dir + 'test_fe_pp.csv')
        return train, test

    def amex_metric(y_true, y_pred):
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, 1].argsort()[::-1]]
        weights = np.where(labels[:,0]==0, 20, 1)
        cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
        top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
        gini = [0,0]
        for i in [1,0]:
            labels = np.transpose(np.array([y_true, y_pred]))
            labels = labels[labels[:, i].argsort()[::-1]]
            weight = np.where(labels[:,0]==0, 20, 1)
            weight_random = np.cumsum(weight / np.sum(weight))
            total_pos = np.sum(labels[:, 0] *  weight)
            cum_pos_found = np.cumsum(labels[:, 0] * weight)
            lorentz = cum_pos_found / total_pos
            gini[i] = np.sum((lorentz - weight_random) * weight)
        return 0.5 * (gini[1]/gini[0] + top_four)

    def amex_metric_np(preds, target):
        indices = np.argsort(preds)[::-1]
        preds, target = preds[indices], target[indices]
        weight = 20.0 - target * 19.0
        cum_norm_weight = (weight / weight.sum()).cumsum()
        four_pct_mask = cum_norm_weight <= 0.04
        d = np.sum(target[four_pct_mask]) / np.sum(target)
        weighted_target = target * weight
        lorentz = (weighted_target / weighted_target.sum()).cumsum()
        gini = ((lorentz - cum_norm_weight) * weight).sum()
        n_pos = np.sum(target)
        n_neg = target.shape[0] - n_pos
        gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)
        g = gini / gini_max
        return 0.5 * (g + d)
    
    def lgb_amex_metric(y_pred, y_true):
      y_true = y_true.get_label()
      return 'amex_metric', amex_metric(y_true, y_pred), True

In [ ]:
lilTrains = []
with tf.device('/device:GPU:0'):
  # for chunk in tqdm(pd.read_csv('test_data.csv', chunksize=500000)):#500k chunks take up less ram and disk
  for chunk in tqdm(pd.read_csv('train_data.csv', chunksize=300000)):
    lilTrains.append(chunk)

19it [03:16, 10.33s/it]


In [ ]:
lilTrains[1].head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
300000,0e00ce4d37a396fa8c3b4e6b29be11eb325333f177a737...,2017-06-16,0.547113,0.008505,0.053466,0.003288,0.003235,0.170432,0.009860,0.003967,NaN,0.417860,0.006212,0.005204,0.395898,0.037823,0.007483,0.429863,0.524211,0.017280,NaN,0.214796,0.054982,0.006150,NaN,0.009342,0.008792,0.206416,0.316510,0.636757,0.083329,0.005162,0.007883,0.032539,0.006642,1.000526,0.005959,0.110615,0.015510,0.748373,0.038812,0.108890,0.018084,0.002617,0.004230,NaN,0.015088,0.006921,0.799743,0.013479,0.002991,0.248565,0.684495,CO,U,0.009597,0.090490,1.004861,0.444048,0.008750,NaN,0.062738,6.0,0.186086,0.007563,0.006599,0.001191,0.008042,0.008946,0.003129,0.012045,0.009484,0.003611,0.030840,NaN,0.003205,0.007581,0.067964,NaN,0.007811,0.000476,0.639203,0.022224,0.001576,0.006641,0.006060,0.000641,NaN,0.008261,0.003602,0.008796,0.500420,0.009466,0.004332,NaN,0.001595,1.006775,0.013704,0.002892,0.003842,0.005695,0.006479,0.001131,0.505988,NaN,0.0,0.002273,0.009207,NaN,0.001929,0.009426,NaN,1,0.000920,0.004855,0.000991,0.000044,0.001197,0.000034,0.003117,0.003262,0.001224,0.003124,0.002495,0.009047,0.005708,0.006896,0.007902,0.007596,0.009547,0.899552,0.131758,0.913199,0.974159,0.001079,0.000484,0.005656,0.000164,NaN,NaN,0.002353,0.008232,0.047539,NaN,1.004327,1.0,NaN,0.003614,NaN,NaN,NaN,1.009849,0.013287,0.007181,0.009785,1.0,0.672799,0.0,-1.0,0.843706,0.841300,0.0,0.748204,0.146553,0.004017,0.046997,0.007432,1.0,0.000488,0.007172,0.001261,0.008713,NaN,0.002852,0.008125,NaN,0.001890,0.001870,NaN,NaN,NaN,NaN,NaN,0.007388,0.004599,0.007238,NaN,0.004326,0.007040,0.005177
300001,0e00ce4d37a396fa8c3b4e6b29be11eb325333f177a737...,2017-07-14,0.547415,0.004299,0.015864,0.706140,0.006674,0.162679,0.003893,0.008383,NaN,0.219995,0.002177,0.000603,0.403372,0.035403,0.001544,0.420933,0.525781,-0.003551,NaN,0.164050,0.016970,0.009292,NaN,0.003590,0.004307,0.200557,0.312517,0.648288,0.220093,0.001305,0.009670,0.006694,0.009667,1.005093,0.009247,0.109277,0.026154,0.945097,0.031630,0.106208,0.021139,0.005968,0.007332,NaN,0.013840,0.022149,0.694828,0.011327,0.003681,0.241647,0.684995,CO,U,0.009914,0.170260,0.008560,0.611689,0.001789,NaN,0.120359,6.0,0.185511,0.005223,0.284979,0.006106,0.006186,0.002926,0.006465,0.010744,0.008713,0.006450,0.007424,NaN,0.000311,0.005133,0.070015,NaN,0.004425,0.002750,0.632460,0.012722,0.000016,0.000768,0.004163,0.002371,NaN,0.009541,0.007027,0.001208,0.500315,0.005834,0.000929,NaN,0.009475,1.007888,0.005246,0.007605,0.005067,0.009507,0.007222,0.008135,0.000195,NaN,0.0,0.000521,0.002342,NaN,0.002263,0.007923,NaN,1,0.007928,0.007674,0.000797,0.009338,0.008467,0.002610,0.002846,0.000027,0.002930,0.001287,0.007164,0.009792,0.009637,0.007235,0.009869,0.006282,0.004877,0.885788,0.140040,0.898510,0.973309,0.004361,0.000885,0.001118,0.004407,NaN,NaN,0.007434,0.006376,0.014245,NaN,1.004745,5.0,NaN,0.006531,NaN,NaN,NaN,1.005830,0.005027,0.003212,0.002845,1.0,0.670279,0.0,-1.0,0.837277,0.844693,0.0,0.740307,0.147996,0.001134,0.053507,0.009726,1.0,0.000453,0.005030,0.009079,0.004793,NaN,0.004855,0.

In [ ]:
lt1 = lilTrains.copy()
lt2 = lilTrains.copy()
lt3 = lilTrains.copy()

In [ ]:
def gpu(train):
  with tf.device('/device:GPU:0'):
    features = train.drop(['customer_ID', 'S_2'], axis=1).columns.to_list()
    cat_features_ = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 
                     'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    train.dropna(axis=1, thresh=(0.7*train.shape[0]), inplace=True)
    cat_features = [x for x in train.columns if x in cat_features_]
    #cat_features redefined

    imputer = SimpleImputer(strategy='constant', fill_value='missing')
    train[cat_features] = imputer.fit_transform(train[cat_features])
    #nans in cat_features have been imputed, cat cols should not have nans

    num_features = [col for col in train.columns if col not in cat_features]
    fillnans(train, num_features[2:], 'mean')
    #nans in remaining num_features have been imputed, no nans at this point

    lol = listOfLists(train)
    b = train[[x for x in lol[0] if x not in cat_features]]
    d = train[[x for x in lol[1] if x not in cat_features]]
    p = train[[x for x in lol[2] if x not in cat_features]]
    r = train[[x for x in lol[3] if x not in cat_features]]
    s = train[[x for x in lol[4] if x not in cat_features]]

    cats = train[cat_features]
    for c in cats.columns:
      encoder = LabelEncoder()
      cats[c] = encoder.fit_transform(cats[c].astype('str'))

    b_flts = floats(b)
    d_flts = floats(d)
    p_flts = floats(p)
    r_flts = floats(r)
    s_flts = floats(s)

    mList = minLst([b[b_flts], d[d_flts], p[p_flts], r[r_flts], s[s_flts]])
    b[b_flts] = b[b_flts] + abs(mList[0]) + 2
    d[d_flts] = d[d_flts] + abs(mList[1]) + 2
    p[p_flts] = p[p_flts] + abs(mList[2]) + 2
    r[r_flts] = r[r_flts] + abs(mList[3]) + 2
    s[s_flts] = s[s_flts] + abs(mList[4]) + 2

    km1 = KMeans(n_clusters=10, random_state=1)
    b_scaled_pca = preprocessor(b[b_flts], km1, 'b')
    for i in range(len(b_scaled_pca.columns)):
      if not isinstance(b_scaled_pca.columns[i], str):
        b_scaled_pca.rename(columns={b_scaled_pca.columns[i]: str(b_scaled_pca.columns[i]+0.5)}, inplace=True)

    km13 = KMeans(n_clusters=14, random_state=99)
    d_scaled_pca = preprocessor(d[d_flts], km13, 'd')
    for i in range(len(d_scaled_pca.columns)):
      if not isinstance(d_scaled_pca.columns[i], str):
        d_scaled_pca.rename(columns={d_scaled_pca.columns[i]: str(d_scaled_pca.columns[i]+34)}, inplace=True)

    scaler = StandardScaler()
    p_scaled = pd.DataFrame(scaler.fit_transform(p[p_flts]), columns = p.columns)
    km29 = KMeans(n_clusters=3, random_state=33)
    km29.fit(p_scaled.values)
    predict3 = km29.predict(p_scaled)
    p_scaled['p_clus'] = pd.Series(predict3, index=p_scaled.index)

    km33 = KMeans(n_clusters=14, random_state=44)
    r_scaled_pca = preprocessor(r[r_flts], km1, 'r')
    for i in range(len(r_scaled_pca.columns)):
      if not isinstance(r_scaled_pca.columns[i], str):
        r_scaled_pca.rename(columns={r_scaled_pca.columns[i]: str(r_scaled_pca.columns[i]+92.5)}, inplace=True)

    km44 = KMeans(n_clusters=14, random_state=55)
    s_scaled_pca = preprocessor(s[s_flts], km1, 's')
    for i in range(len(s_scaled_pca.columns)): 
      if not isinstance(s_scaled_pca.columns[i], str):
        s_scaled_pca.rename(columns={s_scaled_pca.columns[i]: str(s_scaled_pca.columns[i]+119)}, inplace=True)

    t1 = b_scaled_pca.join(d_scaled_pca).join(p_scaled).join(r_scaled_pca).join(s_scaled_pca)
    cats0 = cats.reset_index()
    cats0 = cats0.drop('index', axis=1)
    train1 = cats0.merge(t1, left_index=True, right_index=True)

    train1['customer_ID'] = train['customer_ID'].values

    num_features_ = floats(train1)
    scaler = StandardScaler()
    train1[num_features_] = pd.DataFrame(scaler.fit_transform(train1[num_features_]), columns = num_features_)

    print('Starting train data feature engineering ...')
    train_num_agg = train1.groupby('customer_ID')[num_features_].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)

    for col in train_num_agg:
      if 'last' in col and col.replace('last', 'first') in train_num_agg:
        train_num_agg[col + '_lag_sub'] = train_num_agg[col] - train_num_agg[col.replace('last', 'first')]
        train_num_agg[col + '_lag_div'] = train_num_agg[col] / train_num_agg[col.replace('last', 'first')]

    train_cat_agg = train1.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)

    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
      train_num_agg[col] = train_num_agg[col].astype(np.float32)
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
      train_cat_agg[col] = train_cat_agg[col].astype(np.int32)

    train_diff = get_difference(train1, num_features_)
    fillnans(train_diff, train_diff.columns[:-1], 'min')
    fillnans(train_num_agg, train_num_agg.columns[1:], 'min')
    
    scaler = StandardScaler()
    train_num_agg[train_num_agg.columns[1:]] = pd.DataFrame(scaler.fit_transform
                                                            (train_num_agg[train_num_agg.columns[1:]]), 
                                                            columns = train_num_agg.columns[1:])

    tdiff_feats = floats(train_diff)
    tcat_feats = train_cat_agg.columns[1:]
    tnum_feats = train_num_agg.columns[1:]

    train = train_num_agg.merge(train_cat_agg, 
                            how='inner', 
                            on='customer_ID').merge(train_diff, 
                                                    how='inner', 
                                                    on='customer_ID')

    flts = floats(train)
    mins = minLst([train[flts]])
    train[flts] = train[flts] + abs(np.min(mins)) + 2

    cols = list(train.dtypes[train.dtypes == 'float64'].index)
    for col in tqdm(cols):
      train[col] = train[col].astype(np.float32)
    cols = list(train.dtypes[train.dtypes == 'int64'].index)
    for col in tqdm(cols):
      train[col] = train[col].astype(np.int32)

    fs = floats(train)
    train[fs] = train[fs] + 2
    
    error = True
    while error == True:
      try: 
        pt = PowerTransformer()#default yeo-johnson for train data
        train[fs] = pd.DataFrame(pt.fit_transform(train[fs]), columns=fs)
        error = False
      except:
        print('P_transformer error...')
        train[fs] = train[fs] + 20

    fs = floats(train)
    scaler = StandardScaler()
    train[fs] = pd.DataFrame(scaler.fit_transform(train[fs]), columns = train[fs].columns)

    print('Writing to csv file in google drive...')
    rnum = random.randrange(0, 5000)
    train.to_csv("train_preprocessed_{a}.csv".format(a=rnum))

    return train

In [ ]:
preprocessed = []
for df in lilTrains[1:]:
  preprocessed.append(gpu(df))

# last df broke bc of inf vals

B_41  :  -7.57043206411881
D_145  :  -4.090819515545105
P_4  :  -1.1992691530652153
R_28  :  -0.0916209741848448
S_27  :  -90.48382192636736
Starting train data feature engineering ...


100%|██████████| 24888/24888 [00:19<00:00, 1284.46it/s]


134_diff1  :  -157.72737


0it [00:00, ?it/s]
0it [00:00, ?it/s]


P_transformer error
P_transformer error
Writing to csv file in google drive...
B_41  :  -2.9886274456174244
D_145  :  -3.159135150299555
P_4  :  -1.5196897383880732
R_28  :  -0.0912900541843338
S_27  :  -90.48160243389869
Starting train data feature engineering ...


...

Writing to csv file in google drive...
B_41  :  -2.0493706509581555
D_145  :  -4.301536738672896
P_4  :  -0.9255385689035858
R_28  :  -0.0914650506322711
S_27  :  -90.48062771029984
Starting train data feature engineering ...


100%|██████████| 24926/24926 [00:18<00:00, 1324.01it/s]


134_diff1  :  -686.85864


0it [00:00, ?it/s]
0it [00:00, ?it/s]


Writing to csv file in google drive...
B_41  :  -11.253577374985708
D_145  :  -3.360621687045282
P_4  :  -1.2436255684564108
R_28  :  -0.0916467939220568
S_27  :  -90.48257601844786
Starting train data feature engineering ...


In [ ]:
# count = 0
# for df in preprocessed:
#   df.to_csv("preprocessed_{a}.csv".format(a=count))
#   count += 1

In [ ]:
# len(lilTrains)

0

In [ ]:
# len(lt1)

23

In [ ]:
# lt1[22].head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145
11000000,f7d3a308e06195b6abe4a555e9ef92ae1d8885b9dd46bd...,2018-07-26,0.882480,0.006933,0.007476,0.818229,0.004242,0.228816,0.004892,0.002152,0.003039,0.038642,0.068507,0.003551,0.004204,0.454814,0.568635,0.070639,0.190212,0.025716,1.009381,0.004921,0.326843,0.007702,0.205641,0.649462,0.299603,0.009484,0.006229,1.000437,1.008674,0.002197,0.215911,0.011287,0.006936,0.063573,0.007320,0.003470,0.002792,0.001988,0.566041,0.004853,0.069281,0.009713,0.324068,0.440365,CO,U,0.007960,0.000322,1.001016,0.004872,0.008980,6.0,0.186288,0.005186,0.001190,0.002659,0.008568,0.006452,0.000862,0.007121,0.008069,0.503439,0.016865,0.007835,0.009167,0.002852,0.005652,0.004373,0.008924,0.009904,0.000389,0.008953,0.004938,0.002448,0.009313,0.000059,0.004504,0.007222,0.005101,0.008377,1.002590,0.040703,0.006449,0.004791,0.002640,0.000784,0.004661,0.008412,0.0,0.005659,0.001947,0.003890,0.005936,1,0.006784,0.005676,0.005458,0.001803,0.000274,0.006681,1.005552,0.000040,0.004373,0.004103,0.003598,0.005437,0.004934,0.003236,0.003871,0.004737,0.007772,0.298417,0.133586,0.077051,0.977616,0.000066,0.004643,0.004729,0.003087,0.001639,0.002702,0.006664,1.007093,1.0,0.002955,1.006784,0.031657,0.321339,0.202965,1.0,0.379774,0.0,3.0,0.367134,0.372172,0.0,0.779667,0.436647,0.001027,0.683102,0.003053,1.0,0.005813,0.003020,0.004664,0.003344,0.009635,0.003569,0.009976,0.001077,0.005649,0.008051,0.009371,0.003027,0.007769,0.001229
11000001,f7d3a308e06195b6abe4a555e9ef92ae1d8885b9dd46bd...,2018-08-25,0.884556,0.002221,0.004316,0.818920,0.004244,0.228816,0.003054,0.005577,0.000052,0.045497,0.073958,0.002372,0.005392,0.444807,0.576969,0.040241,0.201527,0.038675,1.000894,0.007306,0.326067,0.007239,0.202718,0.620445,0.300228,0.004334,0.004873,1.003308,1.001014,0.008765,0.215911,0.010394,0.003505,0.055089,0.002471,0.004208,0.009095,0.000838,0.584097,0.007250,0.045501,0.003964,0.326462,0.437226,CO,U,0.001952,0.000316,1.000108,0.002776,0.006253,6.0,0.193917,0.008634,0.005836,0.006697,0.000720,0.002293,0.004507,0.013309,0.005021,0.503484,0.027384,0.000371,0.000941,0.009486,0.002597,0.002776,0.000508,0.007780,0.001146,0.006668,0.004042,0.001565,0.000977,0.008880,0.509197,0.008026,0.009410,0.000888,1.007505,0.041641,0.006176,0.004092,0.004531,0.006666,0.000798,0.006174,0.0,0.005738,0.008219,0.000891,0.008040,1,0.008362,0.004854,0.006005,0.009545,0.004849,0.001291,1.008090,0.004375,0.006528,0.005066,0.006181,0.005280,0.005706,0.008673,0.006999,0.000699,0.003097,0.304293,0.131010,0.082048,0.971586,0.007701,0.000692,0.004835,0.004912,0.009277,0.006729,0.005978,1.006946,1.0,0.004435,1.004091,0.032852,0.321339,0.205077,1.0,0.370996,0.0,3.0,0.370871,0.370693,0.0,0.784036,0.434481,0.007126,0.682454,0.009337,1.0,0.008540,0.001016,0.009474,0.005377,0.006252,0.003935,0.006680,0.004024,0.007438,0.009699,0.006983,0.008642,0.006532,0.006602
11000002,f7d3a308e06195b6abe4a555e9ef92ae1d8885b9dd46bd...,2018-09-25,0.857047,0.001099,0.009851,0.815811,0.006185,0.228816,0.009801,0.000723,0.009673,0.028842,0.072207,0.009181,0.000185,0.452957,0.574449,0.038476,0.196966,0.026045,1.000927,0.001132,0.331924,0.003436,0.201023,0.561258,0.301276,0.002158,0.005694,1.005462,1.004054,0.003837,0.215911,0.006649,0.001143,0

In [ ]:
train = lt1[22]

In [ ]:
features = train.drop(['customer_ID', 'S_2'], axis=1).columns.to_list()
cat_features_ = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 
                  'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# fillnans(train, cat_features_, 'cats')
train.dropna(axis=1, thresh=(0.7*train.shape[0]), inplace=True)
# "['D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_64', 'D_66', 'D_68'] not in index"
cat_features = [x for x in train.columns if x in cat_features_]
#cat_features redefined

imputer = SimpleImputer(strategy='constant', fill_value='missing')
train[cat_features] = imputer.fit_transform(train[cat_features])
#nans in cat_features have been imputed, cat cols should not have nans

num_features = [col for col in train.columns if col not in cat_features]
fillnans(train, num_features[2:], 'mean')
#nans in remaining num_features have been imputed, no nans at this point

In [ ]:
lol = listOfLists(train)
b = train[[x for x in lol[0] if x not in cat_features]]
d = train[[x for x in lol[1] if x not in cat_features]]
p = train[[x for x in lol[2] if x not in cat_features]]
r = train[[x for x in lol[3] if x not in cat_features]]
s = train[[x for x in lol[4] if x not in cat_features]]

cats = train[cat_features]
for c in cats.columns:
  encoder = LabelEncoder()
  cats[c] = encoder.fit_transform(cats[c].astype('str'))

b_flts = floats(b)
d_flts = floats(d)
p_flts = floats(p)
r_flts = floats(r)
s_flts = floats(s)

mList = minLst([b[b_flts], d[d_flts], p[p_flts], r[r_flts], s[s_flts]])
b[b_flts] = b[b_flts] + abs(mList[0]) + 2
d[d_flts] = d[d_flts] + abs(mList[1]) + 2
p[p_flts] = p[p_flts] + abs(mList[2]) + 2
r[r_flts] = r[r_flts] + abs(mList[3]) + 2
s[s_flts] = s[s_flts] + abs(mList[4]) + 2

B_41  :  -3.323347717463657
D_145  :  -3.5341343311060456
P_4  :  -1.0769869075847198
R_28  :  -0.0917077162746496
S_27  :  -91.99839718219324


In [ ]:
km1 = KMeans(n_clusters=10, random_state=1)
b_scaled_pca = preprocessor1(b[b_flts], km1, 'b')
for i in range(len(b_scaled_pca.columns)):
  if not isinstance(b_scaled_pca.columns[i], str):
    b_scaled_pca.rename(columns={b_scaled_pca.columns[i]: str(b_scaled_pca.columns[i]+0.5)}, inplace=True)

km13 = KMeans(n_clusters=14, random_state=99)
d_scaled_pca = preprocessor1(d[d_flts], km13, 'd')
for i in range(len(d_scaled_pca.columns)):
  if not isinstance(d_scaled_pca.columns[i], str):
    d_scaled_pca.rename(columns={d_scaled_pca.columns[i]: str(d_scaled_pca.columns[i]+34)}, inplace=True)

scaler = StandardScaler()
p_scaled = pd.DataFrame(scaler.fit_transform(p[p_flts]), columns = p.columns)
km29 = KMeans(n_clusters=3, random_state=33)
km29.fit(p_scaled.values)
predict3 = km29.predict(p_scaled)
p_scaled['p_clus'] = pd.Series(predict3, index=p_scaled.index)

km33 = KMeans(n_clusters=14, random_state=44)
r_scaled_pca = preprocessor1(r[r_flts], km1, 'r')
for i in range(len(r_scaled_pca.columns)):
  if not isinstance(r_scaled_pca.columns[i], str):
    r_scaled_pca.rename(columns={r_scaled_pca.columns[i]: str(r_scaled_pca.columns[i]+92.5)}, inplace=True)

km44 = KMeans(n_clusters=14, random_state=55)
s_scaled_pca = preprocessor1(s[s_flts], km1, 's')
for i in range(len(s_scaled_pca.columns)): 
  if not isinstance(s_scaled_pca.columns[i], str):
    s_scaled_pca.rename(columns={s_scaled_pca.columns[i]: str(s_scaled_pca.columns[i]+119)}, inplace=True)

In [ ]:
t1 = b_scaled_pca.join(d_scaled_pca).join(p_scaled).join(r_scaled_pca).join(s_scaled_pca)
cats0 = cats.reset_index()
cats0 = cats0.drop('index', axis=1)
train1 = cats0.merge(t1, left_index=True, right_index=True)

train1['customer_ID'] = train['customer_ID'].values
# train1['S_2'] = train['S_2'].values

num_features_ = floats(train1)
scaler = StandardScaler()
train1[num_features_] = pd.DataFrame(scaler.fit_transform(train1[num_features_]), columns = num_features_)

In [ ]:
# train1.isna().sum() -- NO NANS HERE!!

In [ ]:
print('Starting train data feature engineering ...')
train_num_agg = train1.groupby('customer_ID')[num_features_].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace = True)

Starting train data feature engineering ...


In [ ]:
for col in train_num_agg:
  if 'last' in col and col.replace('last', 'first') in train_num_agg:
    train_num_agg[col + '_lag_sub'] = train_num_agg[col] - train_num_agg[col.replace('last', 'first')]
    train_num_agg[col + '_lag_div'] = train_num_agg[col] / train_num_agg[col.replace('last', 'first')]

In [ ]:
train_cat_agg = train1.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)

In [ ]:
# Transform float64 columns to float32
cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
for col in tqdm(cols):
  train_num_agg[col] = train_num_agg[col].astype(np.float32)
# Transform int64 columns to int32
cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
for col in tqdm(cols):
  train_cat_agg[col] = train_cat_agg[col].astype(np.int32)

100%|██████████| 40/40 [00:00<00:00, 794.96it/s]


In [ ]:
# tmp = pd.DataFrame(train_num_agg[train_num_agg.columns[1:]].isna().sum()) -- NANS in std cols

In [ ]:
# tmp = tmp.reset_index()
# tmp.head()

In [ ]:
# tmp[tmp[0] > 0]

In [ ]:
train_num_agg[train_num_agg.columns[1:]].min().min()

-3075266000000000.0

In [ ]:
train_diff = get_difference(train1, num_features_)
fillnans(train_diff, train_diff.columns[:-1], 'min')
fillnans(train_num_agg, train_num_agg.columns[1:], 'min')

100%|██████████| 29591/29591 [00:22<00:00, 1299.41it/s]


In [ ]:
scaler = StandardScaler()
train_num_agg[train_num_agg.columns[1:]] = pd.DataFrame(scaler.fit_transform
                                                        (train_num_agg[train_num_agg.columns[1:]]), 
                                                        columns = train_num_agg.columns[1:])

In [ ]:
# train_num_agg[train_num_agg.columns[1:]].isna().sum().sum() -- NO NANS

In [ ]:
# train_diff[train_diff.columns[:-1]].isna().sum().sum() -- NO NANS

In [ ]:
# train_cat_agg.isna().sum().sum() -- NO NANS

In [ ]:
tdiff_feats = floats(train_diff)
tcat_feats = train_cat_agg.columns[1:]
tnum_feats = train_num_agg.columns[1:]

In [ ]:
train = train_num_agg.merge(train_cat_agg, 
                        how='inner', 
                        on='customer_ID').merge(train_diff, 
                                                how='inner', 
                                                on='customer_ID')

In [ ]:
flts = floats(train)
mins = minLst([train[flts]])
train[flts] = train[flts] + abs(np.min(mins)) + 2

134_diff1  :  -362.45694


In [ ]:
cols = list(train.dtypes[train.dtypes == 'float64'].index)
for col in tqdm(cols):
  train[col] = train[col].astype(np.float32)
# Transform int64 columns to int32
cols = list(train.dtypes[train.dtypes == 'int64'].index)
for col in tqdm(cols):
  train[col] = train[col].astype(np.int32)

0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [ ]:
fs = floats(train)
scaler = StandardScaler()
train[fs] = pd.DataFrame(scaler.fit_transform(train[fs]), columns = train[fs].columns)

In [ ]:
# train.isna().sum().sum()

In [ ]:
mins = minLst([train[fs]])
train[fs] = train[fs] + abs(np.min(mins)) + 2

mins = minLst([train[fs]])
train[fs] = train[fs] + abs(np.min(mins)) + 2

134_diff1  :  -172.01369
134_diff1  :  0


In [ ]:
# train[train.columns[1:]].min().min()

In [ ]:
train[fs] += 2

In [ ]:
# train[train.columns[1:]].min().min()

In [ ]:
train[fs].head()

,0.5_first,0.5_mean,0.5_std,0.5_min,0.5_max,0.5_last,1.5_first,1.5_mean,1.5_std,1.5_min,1.5_max,1.5_last,2.5_first,2.5_mean,2.5_std,2.5_min,2.5_max,2.5_last,3.5_first,3.5_mean,3.5_std,3.5_min,3.5_max,3.5_last,4.5_first,4.5_mean,4.5_std,4.5_min,4.5_max,4.5_last,5.5_first,5.5_mean,5.5_std,5.5_min,5.5_max,5.5_last,6.5_first,6.5_mean,6.5_std,6.5_min,6.5_max,6.5_last,7.5_first,7.5_mean,7.5_std,7.5_min,7.5_max,7.5_last,8.5_first,8.5_mean,8.5_std,8.5_min,8.5_max,8.5_last,9.5_first,9.5_mean,9.5_std,9.5_min,9.5_max,9.5_last,10.5_first,10.5_mean,10.5_std,10.5_min,10.5_max,10.5_last,11.5_first,11.5_mean,11.5_std,11.5_min,11.5_max,11.5_last,12.5_first,12.5_mean,12.5_std,12.5_min,12.5_max,12.5_last,13.5_first,13.5_mean,13.5_std,13.5_min,13.5_max,13.5_last,14.5_first,14.5_mean,14.5_std,14.5_min,14.5_max,14.5_last,15.5_first,15.5_mean,15.5_std,15.5_min,15.5_max,15.5_last,16.5_first,16.5_mean,16.5_std,16.5_min,16.5_max,16.5_last,17.5_first,17.5_mean,17.5_std,17.5_min,17.5_max,17.5_last,18.5_first,18.5_mean,18.5_std,18.5_min,18.5_max,18.5_last,19.5_first,19.5_mean,19.5_std,19.5_min,19.5_max,19.5_last,20.5_first,20.5_mean,20.5_std,20.5_min,20.5_max,20.5_last,21.5_first,21.5_mean,21.5_std,21.5_min,21.5_max,21.5_last,22.5_first,22.5_mean,22.5_std,22.5_min,22.5_max,22.5_last,23.5_first,23.5_mean,23.5_std,23.5_min,23.5_max,23.5_last,24.5_first,24.5_mean,24.5_std,24.5_min,24.5_max,24.5_last,25.5_first,25.5_mean,25.5_std,25.5_min,25.5_max,25.5_last,34_first,34_mean,34_std,34_min,34_max,34_last,35_first,35_mean,35_std,35_min,35_max,35_last,36_first,36_mean,36_std,36_min,36_max,36_last,37_first,37_mean,37_std,37_min,37_max,37_last,38_first,38_mean,38_std,38_min,38_max,38_last,39_first,39_mean,39_std,39_min,39_max,39_last,40_first,40_mean,40_std,40_min,40_max,40_last,41_first,41_mean,41_std,41_min,41_max,41_last,42_first,42_mean,42_std,42_min,42_max,42_last,43_first,43_mean,43_std,43_min,43_max,43_last,44_first,44_mean,44_std,44_min,44_max,44_last,45_first,45_mean,45_std,45_min,45_max,45_last,46_first,46_mean,46_std,46_min,46_max,46_last,47_first,47_mean,47_std,47_min,47_max,47_last,48_first,48_mean,48_std,48_min,48_max,48_last,49_first,49_mean,49_std,49_min,...,57_last_lag_sub,57_last_lag_div,58_last_lag_sub,58_last_lag_div,59_last_lag_sub,59_last_lag_div,60_last_lag_sub,60_last_lag_div,61_last_lag_sub,61_last_lag_div,62_last_lag_sub,62_last_lag_div,63_last_lag_sub,63_last_lag_div,64_last_lag_sub,64_last_lag_div,65_last_lag_sub,65_last_lag_div,66_last_lag_sub,66_last_lag_div,67_last_lag_sub,67_last_lag_div,68_last_lag_sub,68_last_lag_div,69_last_lag_sub,69_last_lag_div,70_last_lag_sub,70_last_lag_div,71_last_lag_sub,71_last_lag_div,72_last_lag_sub,72_last_lag_div,73_last_lag_sub,73_last_lag_div,74_last_lag_sub,74_last_lag_div,75_last_lag_sub,75_last_lag_div,76_last_lag_sub,76_last_lag_div,77_last_lag_sub,77_last_lag_div,78_last_lag_sub,78_last_lag_div,79_last_lag_sub,79_last_lag_div,80_last_lag_sub,80_last_lag_div,81_last_lag_sub,81_last_lag_div,82_last_lag_sub,82_last_lag_div,83_last_lag_sub,83_last_lag_div,P_2_last_lag_sub,P_2_last_lag_div,P_3_last_lag_sub,P_3_last_lag_div,P_4_last_lag_sub,P_4_last_lag_div,92.5_last_lag_sub,92.5_last_lag_div,93.5_last_lag_sub,93.5_last_lag_div,94.5_last_lag_sub,94.5_last_lag_div,95.5_last_lag_sub,95.5_last_lag_div,96.5_last_lag_sub,96.5_last_lag_div,97.5_last_lag_sub,97.5_last_lag_div,98.5_last_lag_sub,98.5_last_lag_div,99.5_last_lag_sub,99.5_last_lag_div,100.5_last_lag_sub,100.5_last_lag_div,101.5_last_lag_sub,101.5_last_lag_div,102.5_last_lag_sub,102.5_last_lag_div,103.5_last_lag_sub,103.5_last_lag_div,104.5_last_lag_sub,104.5_last_lag_div,105.5_last_lag_sub,105.5_last_lag_div,106.5_last_lag_sub,106.5_last_lag_div,107.5_last_lag_sub,107.5_last_lag_div,108.5_last_lag_sub,108.5_last_lag_div,109.5_last_lag_sub,109.5_last_lag_div,110.5_last_lag_sub,110.5_last_lag_div,111.5_last_lag_sub,111.5_last_lag_div,112.5_last_lag_sub,112.5_last_lag_div,119_last_lag_sub,119_last_lag_div,120_last_lag_sub,120_last_lag_div,121_last_l

In [ ]:
pt = PowerTransformer(method='box-cox')
train[fs] = pd.DataFrame(pt.fit_transform(train[fs]), columns=fs) 

In [ ]:
train[fs].head()

,0.5_first,0.5_mean,0.5_std,0.5_min,0.5_max,0.5_last,1.5_first,1.5_mean,1.5_std,1.5_min,1.5_max,1.5_last,2.5_first,2.5_mean,2.5_std,2.5_min,2.5_max,2.5_last,3.5_first,3.5_mean,3.5_std,3.5_min,3.5_max,3.5_last,4.5_first,4.5_mean,4.5_std,4.5_min,4.5_max,4.5_last,5.5_first,5.5_mean,5.5_std,5.5_min,5.5_max,5.5_last,6.5_first,6.5_mean,6.5_std,6.5_min,6.5_max,6.5_last,7.5_first,7.5_mean,7.5_std,7.5_min,7.5_max,7.5_last,8.5_first,8.5_mean,8.5_std,8.5_min,8.5_max,8.5_last,9.5_first,9.5_mean,9.5_std,9.5_min,9.5_max,9.5_last,10.5_first,10.5_mean,10.5_std,10.5_min,10.5_max,10.5_last,11.5_first,11.5_mean,11.5_std,11.5_min,11.5_max,11.5_last,12.5_first,12.5_mean,12.5_std,12.5_min,12.5_max,12.5_last,13.5_first,13.5_mean,13.5_std,13.5_min,13.5_max,13.5_last,14.5_first,14.5_mean,14.5_std,14.5_min,14.5_max,14.5_last,15.5_first,15.5_mean,15.5_std,15.5_min,15.5_max,15.5_last,16.5_first,16.5_mean,16.5_std,16.5_min,16.5_max,16.5_last,17.5_first,17.5_mean,17.5_std,17.5_min,17.5_max,17.5_last,18.5_first,18.5_mean,18.5_std,18.5_min,18.5_max,18.5_last,19.5_first,19.5_mean,19.5_std,19.5_min,19.5_max,19.5_last,20.5_first,20.5_mean,20.5_std,20.5_min,20.5_max,20.5_last,21.5_first,21.5_mean,21.5_std,21.5_min,21.5_max,21.5_last,22.5_first,22.5_mean,22.5_std,22.5_min,22.5_max,22.5_last,23.5_first,23.5_mean,23.5_std,23.5_min,23.5_max,23.5_last,24.5_first,24.5_mean,24.5_std,24.5_min,24.5_max,24.5_last,25.5_first,25.5_mean,25.5_std,25.5_min,25.5_max,25.5_last,34_first,34_mean,34_std,34_min,34_max,34_last,35_first,35_mean,35_std,35_min,35_max,35_last,36_first,36_mean,36_std,36_min,36_max,36_last,37_first,37_mean,37_std,37_min,37_max,37_last,38_first,38_mean,38_std,38_min,38_max,38_last,39_first,39_mean,39_std,39_min,39_max,39_last,40_first,40_mean,40_std,40_min,40_max,40_last,41_first,41_mean,41_std,41_min,41_max,41_last,42_first,42_mean,42_std,42_min,42_max,42_last,43_first,43_mean,43_std,43_min,43_max,43_last,44_first,44_mean,44_std,44_min,44_max,44_last,45_first,45_mean,45_std,45_min,45_max,45_last,46_first,46_mean,46_std,46_min,46_max,46_last,47_first,47_mean,47_std,47_min,47_max,47_last,48_first,48_mean,48_std,48_min,48_max,48_last,49_first,49_mean,49_std,49_min,...,57_last_lag_sub,57_last_lag_div,58_last_lag_sub,58_last_lag_div,59_last_lag_sub,59_last_lag_div,60_last_lag_sub,60_last_lag_div,61_last_lag_sub,61_last_lag_div,62_last_lag_sub,62_last_lag_div,63_last_lag_sub,63_last_lag_div,64_last_lag_sub,64_last_lag_div,65_last_lag_sub,65_last_lag_div,66_last_lag_sub,66_last_lag_div,67_last_lag_sub,67_last_lag_div,68_last_lag_sub,68_last_lag_div,69_last_lag_sub,69_last_lag_div,70_last_lag_sub,70_last_lag_div,71_last_lag_sub,71_last_lag_div,72_last_lag_sub,72_last_lag_div,73_last_lag_sub,73_last_lag_div,74_last_lag_sub,74_last_lag_div,75_last_lag_sub,75_last_lag_div,76_last_lag_sub,76_last_lag_div,77_last_lag_sub,77_last_lag_div,78_last_lag_sub,78_last_lag_div,79_last_lag_sub,79_last_lag_div,80_last_lag_sub,80_last_lag_div,81_last_lag_sub,81_last_lag_div,82_last_lag_sub,82_last_lag_div,83_last_lag_sub,83_last_lag_div,P_2_last_lag_sub,P_2_last_lag_div,P_3_last_lag_sub,P_3_last_lag_div,P_4_last_lag_sub,P_4_last_lag_div,92.5_last_lag_sub,92.5_last_lag_div,93.5_last_lag_sub,93.5_last_lag_div,94.5_last_lag_sub,94.5_last_lag_div,95.5_last_lag_sub,95.5_last_lag_div,96.5_last_lag_sub,96.5_last_lag_div,97.5_last_lag_sub,97.5_last_lag_div,98.5_last_lag_sub,98.5_last_lag_div,99.5_last_lag_sub,99.5_last_lag_div,100.5_last_lag_sub,100.5_last_lag_div,101.5_last_lag_sub,101.5_last_lag_div,102.5_last_lag_sub,102.5_last_lag_div,103.5_last_lag_sub,103.5_last_lag_div,104.5_last_lag_sub,104.5_last_lag_div,105.5_last_lag_sub,105.5_last_lag_div,106.5_last_lag_sub,106.5_last_lag_div,107.5_last_lag_sub,107.5_last_lag_div,108.5_last_lag_sub,108.5_last_lag_div,109.5_last_lag_sub,109.5_last_lag_div,110.5_last_lag_sub,110.5_last_lag_div,111.5_last_lag_sub,111.5_last_lag_div,112.5_last_lag_sub,112.5_last_lag_div,119_last_lag_sub,119_last_lag_div,120_last_lag_sub,120_last_lag_div,121_last_l